In [14]:
import sounddevice as sd
import numpy as np
import tensorflow as tf
import time
import os
import scipy as sp
import scipy.signal

In [58]:
names = {
    0 : 'None',
    1 : 'Джарвис',
}


sd.default.samplerate = 44100
sd.default.channels = 1

rec_duration = 0.1
start_duration = 0.5

model = tf.keras.models.load_model('AudRec_L2_v2.4(12_units).h5')

In [42]:
def norm(x):
    std = np.std(x ,ddof=1)
    return (x - np.mean(x)) / std

def func(_x, _sr=44100):
    _x = np.reshape(_x, len(_x))
    _x = tf.keras.preprocessing.sequence.pad_sequences(
        [_x],
        maxlen=int(_sr * 0.4),
        padding='post',
        truncating='post',
        dtype='float32'
    )[0]
    coeff = sp.signal.firwin(999, [260, 700], fs=_sr, pass_zero=False)
    x_filtered = sp.signal.lfilter(coeff, 1.0, _x)
    x_normalized = x_filtered/x_filtered.max()
    x_squared = np.square(x_normalized)
    splited = np.array_split(x_squared, 20)
    e_parts = np.empty((0))
    for part in splited:
        e_parts = np.append(e_parts, sp.integrate.simps(part))
    return np.reshape(norm(e_parts), (1, 20))

In [59]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                252       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


In [55]:
data = sd.rec(int((start_duration + 0.1) * sd.default.samplerate), blocking=True)

In [71]:
def callback(indata, outdata, frames, time, status):
    global data, model
    print(names[model.predict_classes(func(data))[0]])
    data = data[frames:]
    data = np.append(data, indata, axis=0)

block = int(sd.default.samplerate * rec_duration)
with sd.Stream(blocksize=block, callback=callback):
    sd.sleep(int(1000 * 5))

Джарвис
Джарвис
Привет
Итан
Итан
Итан
Итан
Итан
Итан
Дио
Итан
Дио
Джарвис
Лада
Итан
Итан
Итан
Привет
Лада
Привет
Привет
Итан
Итан
Итан
Итан
Итан
Итан
Итан
Итан
Итан
Итан
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Привет
Итан
Итан
Итан
Итан


In [56]:
sd.play(data)

In [49]:
len(np.reshape(data, (len(data))))

26460

In [40]:
np.reshape(func(data), (1, 20))

array([[-0.69523823, -0.69523807, -0.69523815, -0.61608311, -0.5755576 ,
        -0.5807745 , -0.52167953, -0.60671524, -0.54962544, -0.43167077,
        -0.23957563, -0.33978064, -0.10079884,  1.39306574,  1.92836213,
         2.78512232,  1.20490591,  0.08039432, -0.2321803 , -0.51169436]])

In [57]:
model.predict(func(data))

array([[0.9625995 , 0.03740051]], dtype=float32)

In [4]:
def callback(indata, outdata, frames, time, status):
    outdata[:] = indata
    print(time.currentTime)

block = int(sd.default.samplerate * rec_duration)
stream = sd.Stream(blocksize=block, callback=callback)

In [9]:
stream.start()

In [12]:
stream.active

False

In [10]:
stream.stop()

In [15]:
stream.latency

(0.1, 0.2)

In [62]:
int(1 < 9)

1

In [67]:
a = list(range(10))
print(a)
a = a[5:]
a += list(range(5))
print(a)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[5, 6, 7, 8, 9, 0, 1, 2, 3, 4]


In [68]:
def add(x):
    x += 1
    return x

a = 1
b = add(a)
print(a, b)

1 2
